In [1]:
import functools
from typing import Optional, Sequence, Any, Mapping, Union

import clingo
import clingo.ast
import clingox.backend
import clingox.reify

In [2]:
def solve(ctl: clingo.Control, report=True, report_models=True, report_result=True, sep='\n'):
    models = []
    with ctl.solve(yield_=True) as solve_handle:
        for model in solve_handle:
            symbols = sorted(model.symbols(shown=True))
            if report and report_models:
                print("Answer {}:".format(model.number), end=' ')
                print("{", sep.join(map(str, symbols)), "}", sep=sep)
            models.append(symbols)
        solve_result = solve_handle.get()
        if report and report_result:
            print(solve_result, end='')
            if models:
                print(" {}{}".format(len(models), '' if solve_result.exhausted else '+'))
    return models

In [3]:
p1 = """
a :- k, not b.   b :- not a.   f :- e, not k, not c.
k :- e, not b.   c :- k.       e.
c :- a, b.
"""

In [4]:
class Manipulator(clingo.ast.Transformer):
    loc = clingo.ast.Location(clingo.ast.Position('<string>', 1, 1), clingo.ast.Position('<string>', 1, 1))

In [5]:
class FactToExternal(Manipulator):

    def visit_Rule(self, rule: clingo.ast.AST, external_type: Optional[clingo.ast.AST] = None):
        if not rule.body:
            if external_type is None:
                external_type = clingo.ast.SymbolicTerm(Manipulator.loc, clingo.Function('false'))
            # TODO: check external_type
            return clingo.ast.External(rule.location, rule.head.atom, (), external_type)
        return rule

In [ ]:
class Selector(clingo.ast.Transformer):
    def __init__(self):
        self.selection = []

In [ ]:
class BodySelector(Selector):

    def visit_Rule(self, rule: clingo.ast.AST, head: Union[clingo.ast.AST, clingo.Symbol, None] = None):
        if head is None:
            self.selection.extend(rule.body)
        elif isinstance(head, clingo.Symbol):
            if head.type is clingo.SymbolType.Function:
                if rule.head.name == clin
                self.selection.extend(rule.body)

In [6]:
class BaseCombinator:

    def transform(self, program: str) -> Sequence[clingo.ast.AST]:
        nodes = []
        clingo.ast.parse_string(program, nodes.append)
        return nodes

    def apply(self, program: str, control: Optional[clingo.Control] = None):
        if control is None:
            control = clingo.Control()
        nodes = self.transform(program)
        with clingo.ast.ProgramBuilder(control) as builder:
            for node in nodes:
                builder.add(node)
        return control

In [7]:
class SequenceCombinator(BaseCombinator):

    def __init__(self, manipulators: Sequence[Manipulator], args: Optional[Sequence[Sequence[Any]]] = None,
                 kwargs: Optional[Sequence[Mapping[str, Any]]] = None):
        self._manipulators = manipulators
        self._args = args or tuple(() for _ in manipulators)
        self._kwargs = kwargs or tuple({} for _ in manipulators)

    def _transform_callback(self, stm: clingo.ast.AST, nodes: Optional[Sequence[clingo.ast.AST]] = None) -> None:
        if nodes is None:
            nodes = []
        node = stm
        for i, manipulator in enumerate(self._manipulators):
            args, kwargs = self._args[i], self._kwargs[i]
            node = manipulator.visit(node, *args, **kwargs)
        nodes.append(node)

    def transform(self, program: str) -> Sequence[clingo.ast.AST]:
        nodes = []
        clingo.ast.parse_string(program, functools.partial(self._transform_callback, nodes=nodes))
        return nodes

In [8]:
fact_to_external = SequenceCombinator((FactToExternal(),))
p1t_ast = fact_to_external.transform(p1)
p1t = '\n'.join(map(str, p1t_ast))
print(p1t)

#program base.
a :- k; not b.
b :- not a.
f :- e; not k; not c.
k :- e; not b.
c :- k.
#external e. [false]
c :- a; b.


In [9]:
ctl = clingo.Control()
ctl.configuration.solve.models = 0
ctl.add('base', [], p1)
ctl.ground([('base', [])])

In [10]:
models = solve(ctl, sep=' ')

Answer 1: { b e f }
Answer 2: { a c e k }
SAT 2


In [11]:
ctl.configuration.solve.enum_mode = 'cautious'

In [12]:
cautious_consequence = solve(ctl, sep=' ')[-1]

Answer 1: { b e f }
Answer 2: { e }
SAT 2
